In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_train=pd.read_csv('/kaggle/input/titanic/train.csv')
df_test=pd.read_csv('/kaggle/input/titanic/test.csv')

In [ ]:
df_tr=df_train.copy()
df_te=df_test.copy()

y=df_tr['Survived']
df_tr=df_tr.drop(['PassengerId'],axis=1)
df_te=df_te.drop(['PassengerId'],axis=1)

# Missig Values

In [ ]:
def missing_val(df):
    d={} # dictionary which contains col name and no. of missing values in them
    for col in df.columns:
        l=[]  # list containing count and %ge of missing values for a column
        no_missing_val=df[col].isna().sum()
        l.append(no_missing_val)
        per_missing_val=str((round((100*no_missing_val/df.shape[0]),2)))[0:4]
        l.append(per_missing_val)
        d[col]=l
        
   
    count_v=[]
    for i in range(len(list(d.values()))):
        count_v.append(list(d.values())[i][0])
        
    perc_v=[]
    for i in range(len(list(d.values()))):
        perc_v.append(list(d.values())[i][1])
    
        
    df_missing_val=pd.DataFrame({'Col_Name':list(d.keys()),
                                 'Missing_val_count':count_v,'Missing_val_%':perc_v})
    df_missing_val.sort_values(by='Missing_val_count',axis=0,inplace=True,ascending=False)
    return df_missing_val.style.background_gradient(cmap='RdBu')

    
        
     

# Missing values in training set

In [ ]:
missing_val(df_tr)

# Missing Values in test set

In [ ]:
missing_val(df_te)

In [ ]:
df_tr

# Handling missing values appropriately

In [ ]:
# replacing Missing values in Age,Fare Column with median value
df_tr['Age']=df_tr['Age'].replace(np.nan,df_tr['Age'].median())
df_te['Age']=df_te['Age'].replace(np.nan,df_te['Age'].median())

df_te['Fare']=df_te['Fare'].replace(np.nan,df_te['Fare'].median())

# replacing missing values in Embarked column with mode
df_tr['Embarked']=df_tr['Embarked'].replace(np.nan,str(df_tr['Embarked'].mode()))

#as missing values in Cabin column are more than 75% so treating missing values as a new category

In [ ]:
# as Values in cabin column are  like  c-2 which represent cabin of type c , so considering only first letter

for i in range(df_tr.shape[0]):
    df_tr['Cabin'][i]=str(df_tr['Cabin'][i])[0]
for i in range(df_te.shape[0]):
    df_te['Cabin'][i]=str(df_te['Cabin'][i])[0]

In [ ]:
# Extracting Titles from Names 

'''Name of people doesn't contribute to our model but based on their title we can decide 
whether a person is Male/Female or title can be used for both male and female,and some special
title like capt,col provides some other info like if some one is col. then generally his/her fitness
will be better than other people so more chances of survival'''

for i in range(df_tr.shape[0]):

    df_tr.loc[i,'Title']=(df_tr['Name'][i].split(',')[1].split('.')[0]).strip()
for i in range(df_te.shape[0]):

    df_te.loc[i,'Title']=(df_te['Name'][i].split(',')[1].split('.')[0]).strip()
   

In [ ]:

for i in range(df_tr.shape[0]):
    if str(df_tr['Title'][i])=='Mr' or str(df_tr['Title'][i])=='Master' or str(df_tr['Title'][i])=='Sir':
        df_tr.loc[i,'Title']='Male'
    elif str(df_tr['Title'][i])=='Miss' or str(df_tr['Title'][i])=='Mrs' or str(df_tr['Title'][i])=='Ms':
        df_tr.loc[i,'Title']='Female'
    else:
        df_tr.loc[i,'Title']='Male/Female'
        
        
for i in range(df_te.shape[0]):
    if str(df_te['Title'][i])=='Mr' or str(df_te['Title'][i])=='Master' or str(df_te['Title'][i])=='Sir':
        df_te.loc[i,'Title']='Male'
    elif str(df_te['Title'][i])=='Miss' or  str(df_te['Title'][i])=='Mrs' or str(df_te['Title'][i])=='Ms':
        df_te.loc[i,'Title']='Female'
    else:
        df_te.loc[i,'Title']='Male/Female'
        

In [ ]:
#passengerId, so dropping it
df_tr=df_tr.drop(columns=['Name'])
df_te=df_te.drop(columns=['Name'])

In [ ]:
#libraries for visualizations
import seaborn as sns
import matplotlib.pyplot as plt

# countplots

In [ ]:
import warnings
import math
def count_plot(df):
    categorical=[]
    for col in df.columns:
        if df[col].dtype=='O':
            categorical.append(col)
    
    fig=plt.figure(figsize=(40,25))
    warnings.filterwarnings("ignore")
    for index,col in enumerate(categorical):
        plt.subplot(math.ceil(len(categorical)/1),1,index+1)
        sns.countplot(df[col])
    fig.tight_layout(pad=3.0)
        

In [ ]:
#Sex vs survived
sns.set(style="whitegrid")
plt.figure(figsize=(8,5))
total = float(len(df_tr))
ax = sns.countplot(x="Sex", hue="Survived", data=df_tr)

for p in ax.patches:
    total=len(df_tr)
    percentage = '{:.1f}%'.format(100 * p.get_height()/total)
    x = p.get_x() + p.get_width()
    y = p.get_height()
    ax.annotate(percentage, (x, y),ha='right')
plt.show()

#out of total females about 74%  survived, where as about 19% males out of total males survived


In [ ]:
sns.set(style="whitegrid")
plt.figure(figsize=(10,5))
total = float(len(df_tr))
ax = sns.countplot(x="Pclass", hue="Survived", data=df_tr)


for p in ax.patches:
    percentage = '{:.1f}%'.format(100 * p.get_height()/total)
    x = p.get_x() + p.get_width()
    y = p.get_height()
    ax.annotate(percentage, (x, y),ha='right')
plt.show()

#about 63% of 1st class, about 47% of 2nd class, 24% of 3rd class passengers survived


In [ ]:
# Pclass vs Sex
sns.set(style="whitegrid")
plt.figure(figsize=(15,6))
total = float(len(df_tr))
ax = sns.countplot(x="Pclass", hue="Sex", data=df_tr)

for p in ax.patches:
    percentage = '{:.1f}%'.format(100 * p.get_height()/total)
    x = p.get_x() + p.get_width()
    y = p.get_height()
    ax.annotate(percentage, (x, y),ha='right')
plt.show()

#about 44%,41%,30% females are in 1st,2nd,3rd class respectively 

In [ ]:
# Embarked vs Survived
sns.set(style="whitegrid")
plt.figure(figsize=(15,6))
total = float(len(df_tr))
ax = sns.countplot(x="Embarked", hue="Survived", data=df_tr)

for p in ax.patches:
    percentage = '{:.1f}%'.format(100 * p.get_height()/total)
    x = p.get_x() + p.get_width()
    y = p.get_height()
    ax.annotate(percentage, (x, y),ha='right')
plt.show()

#about 34% of people embarked from Southampton survived

#about 39% of people embarked from Queenstown survived

#about 55% of people embarked from Cherbourg survived

In [ ]:
# title vs Survived
sns.set(style="whitegrid")
plt.figure(figsize=(15,6))
total = float(len(df_tr))
ax = sns.countplot(x="Title", hue="Survived", data=df_tr)

for p in ax.patches:
    percentage = '{:.1f}%'.format(100 * p.get_height()/total)
    x = p.get_x() + p.get_width()
    y = p.get_height()
    ax.annotate(percentage, (x, y),ha='right')
plt.show()


In [ ]:
# Survived vs SibSp
sns.set(style="whitegrid")
plt.figure(figsize=(15,6))
total = float(len(df_tr))
ax = sns.countplot(x="SibSp", hue="Survived", data=df_tr)

for p in ax.patches:
    percentage = '{:.1f}%'.format(100 * p.get_height()/total)
    x = p.get_x() + p.get_width()
    y = p.get_height()
    ax.annotate(percentage, (x, y),ha='right')
plt.show()

# In case of people with one sibling % of survival is more as compared to % of non survival.

In [ ]:
# Survival vs Parch
sns.set(style="whitegrid")
plt.figure(figsize=(15,6))
total = float(len(df_tr))
ax = sns.countplot(x="Parch", hue="Survived", data=df_tr)

for p in ax.patches:
    percentage = '{:.1f}%'.format(100 * p.get_height()/total)
    x = p.get_x() + p.get_width()
    y = p.get_height()
    ax.annotate(percentage, (x, y),ha='right')
plt.show()

# For parents with one child % of survival is more as compared to non survival
# parents with two child half of them survived and half of them died

# histplot

In [ ]:
sns.histplot(df_tr['Age'],color='blue')

In [ ]:
sns.histplot(df_tr['Fare'],color='red')

In [ ]:
for i in df_tr.columns:
    if df_tr[i].isna().sum()!=0:
        print(i)

for i in df_te.columns:
    if df_te[i].isna().sum()!=0:
        print(i)

# boxplot

In [ ]:
sns.boxplot(df_tr['Age'])

In [ ]:
sns.boxplot(df_tr['Fare'])

In [ ]:
for i in df_te.columns:
    if df_te[i].isna().sum()!=0:
        print(i)

In [ ]:
for i in df_te.columns:
    if df_tr[i].isna().sum()!=0:
        print(i,df_tr[i].isna().sum())

In [ ]:
df_tr.columns

In [ ]:
df_tr['SibSp']=df_tr['SibSp'].astype('O')
df_te['SibSp']=df_te['SibSp'].astype('O')


df_tr['Parch']=df_tr['Parch'].astype('O')
df_te['Parch']=df_te['Parch'].astype('O')


In [ ]:
#dealing with missing values
df_tr['Age']=df_tr['Age'].replace(np.nan,df_tr['Age'].median())
df_te['Age']=df_te['Age'].replace(np.nan,df_te['Age'].median())

df_tr['Fare']=df_tr['Fare'].replace(np.nan,df_tr['Fare'].median())
df_te['Fare']=df_te['Fare'].replace(np.nan,df_te['Fare'].median())

#as more than 50% values are missingin cabin column so replacing those with a new category 'No Cabin'
#df_tr['Cabin']=df_tr['Cabin'].replace(np.nan,str(df_tr['Cabin'].mode()))
#df_te['Cabin']=df_te['Cabin'].replace(np.nan,str(df_te['Cabin'].mode()))

df_tr['Embarked']=df_tr['Embarked'].replace(np.nan,str(df_tr['Embarked'].mode()))
df_te['Embarked']=df_te['Embarked'].replace(np.nan,str(df_te['Embarked'].mode()))

In [ ]:
y=df_tr['Survived'].values
df_tr=df_tr.drop('Survived',axis=1)
df=pd.concat([df_tr,df_te])

In [ ]:
sns.regplot(df_tr['Age'],df_tr['Fare'],color='green')

In [ ]:

#encoding categorical data
from sklearn.preprocessing import LabelEncoder
#df['Ticket']=LabelEncoder().fit_transform(df['Ticket'])
#df['Cabin']=LabelEncoder().fit_transform(df['Cabin'])

In [ ]:

for i in range(df_tr.shape[0]):
    if df_tr['Pclass'][i]==1:
        df_tr['Ticket'][i]='first'
    elif df_tr['Pclass'][i]==2:
        df_tr['Ticket'][i]='second'
    elif df_tr['Pclass'][i]==3:
        df_tr['Ticket'][i]='third'
for i in range(df_te.shape[0]):
    if df_te['Pclass'][i]==1:
        df_te['Ticket'][i]='first'
    elif df_te['Pclass'][i]==2:
        df_te['Ticket'][i]='second'
    elif df_te['Pclass'][i]==3:
        df_te['Ticket'][i]='third'


        
        

In [ ]:
df_tr.columns

In [ ]:
df_tr['Pclass']=df_tr['Pclass'].astype('O')
df_te['Pclass']=df_te['Pclass'].astype('O')
#df_tr=df_tr.drop(['Ticket'],axis=1)
#df_te=df_te.drop(['Ticket'],axis=1)

In [ ]:
df_tr.columns

In [ ]:
l=['Pclass','Sex','Age','SibSp','Parch','Ticket', 'Fare', 'Cabin','Embarked']
df_tr['SibSp']=df_tr['SibSp'].astype('O')
df_te['SibSp']=df_te['SibSp'].astype('O')
df1=df_tr[l]
df2=df_te[l]

In [ ]:
df=pd.concat((df1,df2),axis=0)
df.index=[x for x in range(df.shape[0])]

In [ ]:
df.shape

In [ ]:
df=pd.get_dummies(df)
df_tr=df.iloc[0:891,]
df_te=df.iloc[891:,]


In [ ]:
from sklearn.preprocessing import StandardScaler,RobustScaler,MinMaxScaler
from sklearn.model_selection import train_test_split,GridSearchCV
x=df_tr.values
x_tr,x_te,y_tr,y_te=train_test_split(x,y,test_size=.2,random_state=0,stratify=y)

rs=MinMaxScaler()
x_tr=rs.fit_transform(x_tr)
x_te=rs.transform(x_te)
#x=rs.fit_transform(x)

# Model training and performance evaluation

In [ ]:
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,VotingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import StackingClassifier

from sklearn.pipeline import Pipeline

from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

In [ ]:
#building the models
def train_model():
    models={} #this dictionary contains all the models
    models['lr']=LogisticRegression(random_state=0,C= 2, penalty= 'l1', solver= 'saga')
    models['xgb']=XGBClassifier(random_state=0,booster='gbtree',
                                learning_rate=0.1,max_depth= 5,n_estimators=500)
    models['gnb']=GaussianNB(var_smoothing=1e-7)
    
    models['rfc']=RandomForestClassifier(random_state=0,max_depth= 15, 
                                         max_features='auto', n_estimators= 1000)
    models['svc']=SVC(C= 2, degree= 2, kernel= 'linear',probability=True,random_state=0)
    
    models['sgd']=SGDClassifier(random_state=0,loss='log',penalty='elasticnet',
                  alpha=0.001,learning_rate='optimal',eta0=1e-05,l1_ratio=0.05,max_iter=1000)
    
    models['knn']=KNeighborsClassifier(n_neighbors=5,weights='uniform'
                                       ,algorithm='auto',p=1,leaf_size=10)
    
    models['dtc']=DecisionTreeClassifier(criterion='gini',max_depth=13,max_features='auto'
                                         ,min_samples_leaf=3
                           ,min_samples_split=3,splitter='best',random_state=0)
    models['gbc']=GradientBoostingClassifier(random_state=0)
    
    return models

model=train_model()
    

In [ ]:
def evaluation(y,y_pred):
    accuracy=accuracy_score(y,y_pred)
    confusion_matrix_=confusion_matrix(y,y_pred)
    print(accuracy_score)
    print(confusion_matrix_)
    print(classification_report(y,y_pred))
    return accuracy


In [ ]:
l=[]
for k,v in model.items():
    m=v
    print(k)
    m.fit(x_tr,y_tr)
    y_pred=m.predict_proba(x_te)[:,1]
    a=evaluation(y_te,m.predict(x_te))
    l.append(a)
    

In [ ]:
estimators=[('knn',model['knn']),('svc',model['svc']),
            ('sgd',model['sgd']),('rfc',model['rfc']),
            ('dtc',model['dtc']),('lr',model['lr']),('gbc',model['gbc']),('xgb',model['xgb'])]
#stack_model=StackingClassifier(estimators=estimators, final_estimator=model['xgb'], stack_method='auto', passthrough=True,verbose=5,n_jobs=-1)
#stack_model.fit(x,y)


In [ ]:

#evaluation(y,stack_model.predict(x))

In [ ]:
voating=VotingClassifier(estimators = estimators, voting ='hard',n_jobs=-1)
voating.fit(x,y)

In [ ]:
help(VotingClassifier)

In [ ]:
evaluation(y,voating.predict(x))

In [ ]:
df=pd.read_csv('/kaggle/input/titanic/test.csv')

df_te=rs.transform(df_te)
PassengerId=df['PassengerId']
y=voating.predict(df_te)

In [ ]:
df_csv=pd.DataFrame({'PassengerId':PassengerId,'Survived':y})
df_csv.to_csv('Submission_titanic_voating_6.csv',index=False)

In [ ]:
pr()

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier(algorithm='auto', p= 1, weights= 'uniform',n_neighbors=7)
knn.fit(x_tr,y_tr)
y_pr=knn.predict_prob(x_te)

print(classification_report(y_te,y_pr))
print(accuracy_score(y_te,y_pr))
confusion_matrix(y_te,y_pr)



#Hyper parameter tuning
#params={'algorithm' : ['auto', 'ball_tree', 'kd_tree', 'brute'],'p':[1,2],'weights':['uniform','distance'],'n_neighbors':[5,7,9,10,12,14,15,16,17,3,4,6,8,11,13]}
#gcv=GridSearchCV(knn,params,cv=3,n_jobs=24,verbose=5)
#gcv.fit(x,y)



In [ ]:
gcv.best_params_

In [ ]:
y_pr=rfc.predict(x_te)

print(classification_report(y_te,y_pr))
print(accuracy_score(y_te,y_pr))
confusion_matrix(y_te,y_pr)

In [ ]:
help(rfc)

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.tree import DecisionTreeClassifier




estimator=[('rf',RandomForestClassifier(random_state=0,)),('lr',LogisticRegression(solver='liblinear')),('xgb',XGBClassifier(random_state=0)),('dt',DecisionTreeClassifier())]

stc=StackingClassifier(estimators=estimator,final_estimator=XGBClassifier(random_state=0),cv=5,passthrough=True)
stc.fit(x_tr,y_tr)

y_pr=stc.predict(x_te)

print(classification_report(y_te,y_pr))
print(accuracy_score(y_te,y_pr))
confusion_matrix(y_te,y_pr)

In [ ]:
g = sns.FacetGrid(df_tr,  col='Survived', aspect=2 ,sharey=False,sharex=False)
g.map_dataframe(sns.histplot, x='Age', color='xkcd:magenta',bins=7)
#g.set_titles(row_template='{row_name}', col_template='{col_name} Penguins')
g.set_axis_labels('Age', "Count")

plt.tight_layout()
#plt.savefig('15facetgrid_thumb.png', dpi=200);

# in 1st class 

In [ ]:
import graphviz  
from sklearn.tree import export_graphviz  
dot_data= export_graphviz(model.estimators_[10], out_file = None, 
                    feature_names = df_tr.columns,  
                      class_names = ['0', '1'],  
                      filled = True, rounded = True,  
                      special_characters = True, impurity = True)  
graph = graphviz.Source(dot_data, format='png')  
graph  

In [ ]:
def barPerc(df,xVar,yVar,ax):
    '''
    barPerc(): Add percentage for hues to bar plots
    args:
        df: pandas dataframe
        xVar: (string) X variable 
        yVar: string hue
        ax: Axes object (for Seaborn Countplot/Bar plot or
                         pandas bar plot)
    '''
    # 1. how many X categories
    ##   check for NaN and remove
    numX=len([x for x in df[xVar].unique() if x==x])
   

    # 2. The bars are created in hue order, organize them
    bars = ax.patches
    
    ## 2a. For each X variable
    for ind in range(numX):
        ## 2b. Get every hue bar
        ##     ex. 8 X categories, 4 hues =>
        ##    [0, 8, 16, 24] are hue bars for 1st X category
        hueBars=bars[ind:][::numX]
       
        
        ## 2c. Get the total height (for percentages)
        total = sum([x.get_height() for x in hueBars])

        # 3. Print the percentage on the bars
        for bar in hueBars:
            ax.text(bar.get_x() + bar.get_width()/2.,
                    bar.get_height(),
                    f'{bar.get_height()/total:.0%}',
                    ha="center",va="bottom")

In [ ]:
ax=sns.countplot(x='Pclass',hue='Survived',data=df_tr)
barPerc(df_tr,'Pclass','Survived',ax)

In [ ]:
help(ax.patches)